<a href="https://colab.research.google.com/github/chacha715/ESAA/blob/main/%EB%B3%91%EC%9B%90_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
#train = pd.read_csv('/content/drive/MyDrive/ESAA/YB 1조/9565_hospital_data/train.csv')
#test = pd.read_csv('/content/drive/MyDrive/ESAA/YB 1조/9565_hospital_data/test.csv')

train = pd.read_csv('/content/drive/MyDrive/ESAA/YB 1조/train 지역변수 추가.csv')
test = pd.read_csv('/content/drive/MyDrive/ESAA/YB 1조/test 지역변수 추가.csv')



```
# EDA 전처리
```



In [13]:
#train.drop(['inst_id','sgg'],axis=1,inplace=True)

train.dropna(subset=['instkind'],axis=0,inplace=True)
test.dropna(subset=['instkind'],axis=0,inplace=True)

In [14]:
#train.dropna(subset=['instkind'],axis=0,inplace=True)

train['ownerChange'].mode()[0]
train.loc[train['ownerChange'].isna(),'ownerChange'] = train['ownerChange'].mode()[0]
train['ownerChange'].isna().sum()

test['ownerChange'].mode()[0]
test.loc[test['ownerChange'].isna(),'ownerChange'] = test['ownerChange'].mode()[0]
test['ownerChange'].isna().sum()

0

In [16]:
train.dropna(subset=['salescost1'],axis=0,inplace=True)
test.dropna(subset=['salescost1'],axis=0,inplace=True)

In [17]:
train.groupby('instkind')['bedCount'].mean()
train['bedCount'] = train['bedCount'].fillna(train.groupby('instkind')['bedCount'].transform('mean')) 
train['bedCount'].isnull().sum()

test.groupby('instkind')['bedCount'].mean()
test['bedCount'] = test['bedCount'].fillna(test.groupby('instkind')['bedCount'].transform('mean')) 
test['bedCount'].isnull().sum()

1

In [18]:
for i in range(0,len(train["OC"])):
  if train.iloc[i,0] == "open":
    train.iloc[i, 0]=1
  else:
    train.iloc[i, 0]=0 

In [19]:
round(train['ownerChange'].value_counts(normalize=True),3)
train['ownerChange'] = train['ownerChange'].fillna(pd.Series(np.random.choice(['same','change'], p=[0.85,0.15],size = 300))) 



```
# 파생변수 생성
```



In [21]:
#매출액순이익율=당기순이익/자기자본*100

train['매출액순이익율'] = train.profit1/train.revenue1*100

In [22]:
#배당성향=배당금/당기순이익
#배당금=당기순이익-이익잉여금

train['배당성향'] = (train.profit1-train.surplus1)/train.profit1

In [23]:
#이자보상비율=영업이익/이자비용*100
#영업손익은 매출액 등의 영업수익으로부터 매출원가ㆍ판매비와관리비 등의 영업비용을 차감한 개념 (영업 외 이익과 비용도 차감)

train['이자보상비율'] = (train.revenue1-train.salescost1-train.sga1-train.noi1-train.noe1)/train.interest1*100

In [24]:
##은서

#재고자산회전율=매출원가/평균재고자산
train['재고자산회전율']=train.salescost1/((train.inventoryAsset1+train.inventoryAsset2)/2)

#자기자본비율=자기자본/총자본*100
train['자기자본비율']=train.netAsset1/(train.debt1+train.netAsset1)*100

In [25]:
##현서

#유동비율 = 유동자산/유동부채 *100
train['유동비율']=(train['liquidAsset1']/train['liquidLiabilities1'])*100
#당좌비율 = (유동자산-재고자산)/유동부채 *100
train['당좌비율']=(train['quickAsset1']/train['liquidLiabilities1'])*100
#부채비율 = 부채/자본 *100
train['부채비율']=(train['debt1']/(train['debt1']+train['netAsset1']))*100

In [26]:
##재령

train['매출액증가율']=(train['revenue1']/train['revenue2'])*100-100
train['총자산증가율']=(train['netAsset1']+train['debt1'])/(train['netAsset2']+train['debt2'])*100 -100
train['ROE']=(train['profit1']/train['netAsset1'])*100

In [20]:
##미수

train.loc[train.bedCount>300,'hospital_size']=1
train.loc[train.bedCount<=300,'hospital_size']=0

In [32]:
train.describe()

,openDate,bedCount,revenue1,salescost1,sga1,salary1,noi1,noe1,interest1,ctax1,...,이자보상비율,재고자산회전율,자기자본비율,유동비율,당좌비율,부채비율,매출액증가율,총자산증가율,ROE,hospital_size
count,2.920000e+02,292.000000,2.920000e+02,2.920000e+02,2.920000e+02,2.920000e+02,2.920000e+02,2.920000e+02,2.920000e+02,2.920000e+02,...,292.000000,266.000000,264.000000,267.000000,267.000000,264.000000,289.000000,264.000000,292.000000,292.000000
mean,2.005035e+07,142.673888,1.292482e+10,2.021726e+09,1.036622e+10,5.672605e+09,2.704924e+08,5.130304e+08,1.875221e+08,7.874196e+07,...,NaN,inf,38.723885,inf,inf,61.276115,inf,inf,NaN,0.078767
std,8.859836e+04,117.585777,2.045719e+10,7.472162e+09,1.495329e+10,8.091010e+09,8.036068e+08,1.061911e+09,2.741262e+08,2.489944e+08,...,NaN,NaN,42.123776,NaN,NaN,42.123776,NaN,NaN,NaN,0.269837
min,1.978012e+07,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-inf,0.000000,-437.041102,5.153869,5.153869,0.000000,-100.000000,-24.096703,-inf,0.000000
25%,2.001117e+07,52.000000,3.261234e+09,0.000000e+00,2.825778e+09,1.631012e+09,8.179622e+06,8.007077e+07,5.172361e+07,0.000000e+00,...,-314.944860,1.116397,26.705177,41.047893,38.650658,44.939843,-1.397761,-1.258887,-0.007859,0.000000
50%,2.007116e+07,133.000000,5.554592e+09,2.112576e+08,4.717490e+09,2.664001e+09,4.482983e+07,1.839620e+08,1.098159e+08,1.134180e+06,...,4.822876,11.483622,41.252349,81.437475,78.845424,58.747651,3.724451,2.644641,3.478714,0.000000
75%,2.011087e+07,185.250000,1.286315e+10,9.106278e+08,1.078348e+10,6.380456e+09,2.057630e+08,4.221954e+08,2.251664e+08,4.445450e+07,...,211.103501,33.901424,55.060157,199.245287,192.412602,73.294823,12.852503,9.923170,9.941502,0.000000
max,2.017061e+07,656.000000,1.510000e+11,9.850332e+10,1.030000e+11,6.403559e+10,9.144171e+09,8.686380e+09,2.841475e+09,2.808883e+09,...,inf,inf,100.000000,inf,inf,537.041102,inf,inf,inf,1.000000


In [34]:
train['profit1'].describe()

count    2.920000e+02
mean     2.117841e+08
std      1.136284e+09
min     -4.696701e+09
25%     -5.420698e+05
50%      8.208208e+07
75%      2.946849e+08
max      7.475427e+09
Name: profit1, dtype: float64

In [36]:
train['revenue1'].isnull().sum()

0